In [6]:
import os
import tushare as ts
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

import sys
sys.path.append('..')  # 将父文件夹添加到 Python 路径中，不然找不到下面的包

from utilities.tushare_limit_up import *

以下代码是utilities.tushare_limit_up中generate_limit_up函数的展开

In [7]:
if os.getenv("ENV") != "product":
    # 在开发环境中加载 .env 文件
    load_dotenv(override=True)

# 初始化pro接口
token = os.getenv('TUSHARE_API_KEY')
pro = ts.pro_api(token)

# Generate today's date in "yyyymmdd" format
today_str = datetime.now().strftime("%Y%m%d")
cal_date = getLatestTradeData(pro, today_str)

cal_date

'20240313'

# 以下是getLimitUpData的展开
limit_list_d接口获取到的数据是收盘价等于涨停板的股票
daily中进行筛选获取涨幅大于10%的股票
两者取股票代码的并集

然后对基础的数据进行合并

In [9]:
limit_up_df = pro.limit_list_d(**{ "trade_date": cal_date, "limit_type": "U", }, fields=[ "trade_date", "ts_code", "limit_amount", "fd_amount", "first_time", "last_time", "open_times", "up_stat", "limit_times", "limit" ])
daily_df = pro.daily(**{"trade_date": cal_date}, fields=[ "ts_code", "trade_date", "open", "high", "low", "close", "pre_close", "change", "pct_chg", "vol", "amount" ])

stock_basic_df = pro.stock_basic(**{"list_status": 'L'}, fields=['ts_code','name','industry'])
daily_basic_df = pro.daily_basic(**{"trade_date": cal_date}, fields=['ts_code','trade_date','circ_mv','total_mv','turnover_rate','turnover_rate_f','pe','pb'])


In [ ]:
tenpct_df = daily_df[daily_df['pct_chg'] >= 10]

# 合并tenpct_df和limit_up_df
union_tscode = pd.concat([tenpct_df['ts_code'], limit_up_df['ts_code']]).drop_duplicates()
union_tscode_df = daily_df[daily_df['ts_code'].isin(union_tscode)]
merged_df = pd.merge(union_tscode_df, stock_basic_df, on=['ts_code'], how='left')
merged_df = pd.merge(merged_df, daily_basic_df, on=['ts_code'], how='left')
merged_df = pd.merge(merged_df, limit_up_df, on=['ts_code'], how='outer')

df = merged_df